In [29]:
import pandas as pd
import numpy as np
import torch
from skimage import io

In [30]:
df_male = pd.read_csv("./male_yellow_images.csv")
df_female = pd.read_csv("./female_yellow_images.csv")

In [31]:
df_male.head()

,Email,Filename,Rating,original Rating
0,1,mty1.jpg,4,NaN
1,2,mty1.jpg,4,4.0
2,3,mty1.jpg,5,NaN
3,4,mty1.jpg,3,NaN
4,5,mty1.jpg,4,NaN


In [32]:
df_male.values.shape

(120000, 4)

In [33]:
landmarks = torch.load("./fan_landmark/saved_images_preds.tch")

In [34]:
def get_median(num_people, df_people, path, prefix):
    target_columns = ['Filename', 'Rating', 'Female', 'Male', 'Yellow','White'] + [str(i) for i in range(68*2)]
    df_rt = pd.DataFrame(columns = target_columns)
    for i in range(num_people):
        filename = "%s%d.jpg"%(prefix, i+1)
#         rating_mean = int(df_people.loc[df_people['Filename'] == filename, 'Rating'].mean() + 0.49) - 1
        rating_mean = df_people.loc[df_people['Filename'] == filename, 'Rating'].mean() - 1
        male = 1 if prefix[0] == 'm' else 0
        female = 1 if prefix[0] == 'f' else 0
        yellow = 1 if prefix[2] == 'y' else 0
        white = 1 if prefix[2] == 'w' else 0
        landmark = list(landmarks['./Images/'+filename].view(-1).numpy().astype(np.int))
        row = pd.Series([path + filename, rating_mean, female, male, yellow, white] + landmark, index=target_columns)
        df_rt = df_rt.append(row, ignore_index=True)
    return df_rt

In [35]:
num_male = 2000
df_mty = get_median(num_male, df_male, "./ImageFolder/", "mty")

In [36]:
df_mty.head()

,Filename,Rating,Female,Male,Yellow,White,0,1,2,3,...,126,127,128,129,130,131,132,133,134,135
0,./ImageFolder/mty1.jpg,2.950000,0,1,1,0,94,160,96,184,...,183,252,203,247,183,250,175,252,167,250
1,./ImageFolder/mty2.jpg,2.600000,0,1,1,0,69,135,72,162,...,194,235,216,233,194,235,183,235,173,235
2,./ImageFolder/mty3.jpg,3.050000,0,1,1,0,39,132,43,167,...,181,269,204,269,181,273,169,273,157,273
3,./ImageFolder/mty4.jpg,2.316667,0,1,1,0,89,162,92,187,...,190,236,203,239,190,236,181,236,173,236
4,./ImageFolder/mty5.jpg,1.300000,0,1,1,0,78,183,80,210,...,183,267,205,267,183,267,175,267,164,267


In [37]:
df_female.shape

(120000, 4)

In [38]:
num_female = 2000
df_fty = get_median(num_female, df_female, "./ImageFolder/", "fty")

In [39]:
con = pd.concat([df_mty, df_fty])
con.head()

,Filename,Rating,Female,Male,Yellow,White,0,1,2,3,...,126,127,128,129,130,131,132,133,134,135
0,./ImageFolder/mty1.jpg,2.950000,0,1,1,0,94,160,96,184,...,183,252,203,247,183,250,175,252,167,250
1,./ImageFolder/mty2.jpg,2.600000,0,1,1,0,69,135,72,162,...,194,235,216,233,194,235,183,235,173,235
2,./ImageFolder/mty3.jpg,3.050000,0,1,1,0,39,132,43,167,...,181,269,204,269,181,273,169,273,157,273
3,./ImageFolder/mty4.jpg,2.316667,0,1,1,0,89,162,92,187,...,190,236,203,239,190,236,181,236,173,236
4,./ImageFolder/mty5.jpg,1.300000,0,1,1,0,78,183,80,210,...,183,267,205,267,183,267,175,267,164,267


In [40]:
con.tail()

,Filename,Rating,Female,Male,Yellow,White,0,1,2,3,...,126,127,128,129,130,131,132,133,134,135
1995,./ImageFolder/fty1996.jpg,0.600000,1,0,1,0,31,116,40,154,...,181,267,210,267,181,267,167,272,158,267
1996,./ImageFolder/fty1997.jpg,0.433333,1,0,1,0,80,143,80,170,...,171,242,194,245,171,251,162,251,152,251
1997,./ImageFolder/fty1998.jpg,1.733333,1,0,1,0,83,160,85,184,...,192,252,211,250,192,252,183,252,173,252
1998,./ImageFolder/fty1999.jpg,1.733333,1,0,1,0,75,164,81,191,...,206,266,225,263,206,266,196,269,183,269
1999,./ImageFolder/fty2000.jpg,2.550000,1,0,1,0,88,154,91,179,...,186,244,208,239,186,244,178,244,167,244


In [41]:
con['Rating'].max(), con['Rating'].min()

(3.75, 0.016666666666666607)

In [ ]:
import torchvision.transforms as transforms

In [ ]:
tr = transforms.RandomResizedCrop(224)

In [ ]:
for path in con['Filename']:
    img = io.imread(f"./ImageFolder/{path[path.rfind('/') + 1:]}")
    lanimg = np.zeros((1, 350, 350)).astype(np.int8)
    for x, y in landmarks[f"./Images/{path[path.rfind('/') + 1:]}"]:
        x, y = x.numpy().astype(np.int), y.numpy().astype(np.int)
        x = np.clip(x, 0, 349)
        y = np.clip(y, 0, 349)
        lanimg[0, y, x] = 1
    np.save(f"{path[:path.rfind('.')]}.landmark", lanimg)

In [ ]:
lanimg.shape

In [ ]:
lanimg[:,:,0]

In [ ]:
a = con.iloc[0]
path = a['Filename']
img = io.imread(path)
io.imshow(img)

In [ ]:
io.imsave(conimg

In [ ]:
io.imshow(conimg)

In [42]:
con.to_csv("./median.csv", index=False)
con.head()

,Filename,Rating,Female,Male,Yellow,White,0,1,2,3,...,126,127,128,129,130,131,132,133,134,135
0,./ImageFolder/mty1.jpg,2.950000,0,1,1,0,94,160,96,184,...,183,252,203,247,183,250,175,252,167,250
1,./ImageFolder/mty2.jpg,2.600000,0,1,1,0,69,135,72,162,...,194,235,216,233,194,235,183,235,173,235
2,./ImageFolder/mty3.jpg,3.050000,0,1,1,0,39,132,43,167,...,181,269,204,269,181,273,169,273,157,273
3,./ImageFolder/mty4.jpg,2.316667,0,1,1,0,89,162,92,187,...,190,236,203,239,190,236,181,236,173,236
4,./ImageFolder/mty5.jpg,1.300000,0,1,1,0,78,183,80,210,...,183,267,205,267,183,267,175,267,164,267


In [ ]:
df_mty.to_csv("./mty_mean_3.csv",index=False)


In [ ]:
df_fty.to_csv("./fty_mean.csv", index=False)